In [1]:
import numpy as np

with open('input.txt') as f:
    data = f.read()
data = "".join(format(int(x, 16),'04b') for x in data)

In [2]:
#old part 1, edited a bit for part 2

def solve_packet(s):
    if not '1' in s:
        return 0, ''
    version = s[0:3]
    type_id = s[3:6]
    vsum = int(version, 2)
    out = []
    if type_id == "100":
        ind = 6
        while s[ind] == '1':
            out += [s[ind+1:ind+5]]
            ind += 5
        out += s[ind+1:ind+5]
        _vsum, _out = solve_packet(s[ind+5:])
        vsum += _vsum
        out  += _out
        out = "".join(out)
        
    else:
        len_type_id = s[6:7]
        if len_type_id == '0':
            L = int(s[7:22],2)
            _vsum, _out = solve_packet(s[22:])
            vsum += _vsum
            out  += _out
        elif len_type_id =='1':
            n = int(s[7:18],2)
            _vsum, _out = solve_packet(s[18:])
            vsum += _vsum
            out  += _out
    return vsum, out
tststr = "8A004A801A8002F478"
tststr = "".join(format(int(x, 16),'04b') for x in tststr)
solve = solve_packet(tststr)
print(f"solve: {solve}, packet sum: {solve[0]}, val: {solve[1]}")

solve: (16, ['1', '1', '1', '1']), packet sum: 16, val: ['1', '1', '1', '1']


In [3]:
b2i = lambda x: int(x, 2)
i2b = lambda x: format(x, '04b')
dashes = ''
#probably is an easier implementation to use a generator, but meh
def parse(s):
    global vsum
    global ind
    global dashes
    #get version
    vsum += b2i(s[ind:ind+3])
    ind += 3
    #get type id
    tid = s[ind:ind+3]
    ind += 3
    
    #if tid is 4, do fancy concat thingy
    if tid == '100':
        out = []
        while True:
            end, *bits = s[ind:ind+5]
            ind += 5
            out += bits
            if end == '0':
                break
        #print(dashes, b2i("".join(out)))
        return b2i("".join(out))
    #get length type id:
    lid = s[ind]
    ind += 1
    sp_vals = []
    #if lid is 0, length is next 15 bits, so parse that length of the string
    old_dashes = dashes
    dashes = dashes + ' - '
    if lid == '0':
        L = b2i(s[ind:ind+15])
        ind += 15
        old_ind = ind
        while old_ind + L > ind:
            sp_vals += [parse(s)]
    #else number of sub vals is next 11 bits, so parse until youve gotten that many values
    else:
        n = b2i(s[ind:ind+11])
        ind += 11
        sp_vals = [parse(s) for i in range(n)]
    dashes = old_dashes
    #print(dashes, tid, sp_vals)
    
    #if tid is 0 take sum
    if tid == '000':
        return sum(sp_vals)
    #if tid is 1 take prod
    if tid == '001':
        #coded in python because numpy was overflowing :(
        p = 1
        for x in sp_vals:
            p *= x
        return int(p)
    #if tid is 2 take min
    if tid == '010':
        return min(sp_vals)
    #if tid is 3 take max
    if tid == '011':
        return max(sp_vals)
    #if tid is 5 return >
    if tid == '101':
        return int(sp_vals[0] > sp_vals[1])
    #if tid is 6 return <
    if tid == '110':
        return int(sp_vals[0] < sp_vals[1])
    #if tid is 7 return ==
    if tid == '111':
        return int(sp_vals[0] == sp_vals[1])

In [4]:
vsum = 0
ind = 0
solve = parse(data)
print(f"vsum: {vsum}")
print(f"solve: {(solve)}")

vsum: 969
solve: 124921618408
